# Data Cleaning: Extract Loans with Description

## 目标
1. 从 loan.csv 中提取所有有 desc（描述）的数据行
2. 分析数值型变量（quantitative variables）质量
3. 保存清理后的数据为 loan_with_desc.csv
4. 生成数据质量报告

---

In [ ]:
# Cell 1: 导入库和设置
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# 设置图表样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ 库导入成功")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

In [ ]:
# Cell 2: 加载数据
print("正在加载 loan.csv...")
print("(这可能需要1-2分钟，数据集约1.1GB)\n")

df = pd.read_csv('../../data/loan.csv', low_memory=False)

print("="*80)
print("原始数据集基本信息")
print("="*80)
print(f"总行数: {len(df):,}")
print(f"总列数: {len(df.columns)}")
print(f"\n内存使用: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n前5行预览:")
df.head()

In [ ]:
# Cell 3: 过滤有 desc 的数据
print("="*80)
print("步骤 1: 过滤有 desc 的数据")
print("="*80)

# 检查 desc 列的基本情况
print(f"\ndesc 列中非空值数量: {df['desc'].notna().sum():,}")
print(f"desc 列中空值数量: {df['desc'].isna().sum():,}")

# 过滤条件：desc 非空且去除空格后长度 > 1
df_with_desc = df[
    df['desc'].notna() & 
    (df['desc'].astype(str).str.strip().str.len() > 1)
].copy()

print(f"\n✓ 过滤后的数据集:")
print(f"  - 行数: {len(df_with_desc):,}")
print(f"  - 占原数据集比例: {len(df_with_desc)/len(df)*100:.2f}%")
print(f"  - 列数: {len(df_with_desc.columns)} (保持不变)")

# 显示 desc 字段的统计信息
desc_lengths = df_with_desc['desc'].astype(str).str.len()
print(f"\ndesc 文本长度统计:")
print(f"  - 平均长度: {desc_lengths.mean():.0f} 字符")
print(f"  - 中位数长度: {desc_lengths.median():.0f} 字符")
print(f"  - 最短: {desc_lengths.min()} 字符")
print(f"  - 最长: {desc_lengths.max()} 字符")

# 显示几个样本
print("\n样本描述 (前3个):")
print("="*80)
for i, desc in enumerate(df_with_desc['desc'].head(3), 1):
    desc_str = str(desc)
    preview = desc_str[:200] + "..." if len(desc_str) > 200 else desc_str
    print(f"\n[样本 {i}] (长度: {len(desc_str)} 字符)")
    print(preview)
    print("-"*80)

In [ ]:
# Cell 4: 识别和分类所有变量
print("="*80)
print("步骤 2: 变量分类与清理")
print("="*80)

# 1. 初步分类
numeric_cols_raw = df_with_desc.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols_raw = df_with_desc.select_dtypes(include=['object']).columns.tolist()

print(f"\n初步分类:")
print(f"  - 数值型变量: {len(numeric_cols_raw)} 个")
print(f"  - 类别型变量: {len(categorical_cols_raw)} 个")

# 2. 清理百分比字符串（转换为数值）
print(f"\n清理百分比字符串...")
percent_cols = ['int_rate', 'revol_util', 'sec_app_revol_util']
cleaned_count = 0

for col in percent_cols:
    if col in df_with_desc.columns and df_with_desc[col].dtype == object:
        df_with_desc[col] = pd.to_numeric(
            df_with_desc[col].astype(str).str.strip().str.rstrip('%'),
            errors='coerce'
        )
        cleaned_count += 1
        print(f"  ✓ {col}: 转换为数值型")

print(f"\n共清理了 {cleaned_count} 个百分比列")

# 3. 去除类别型变量的空格
print(f"\n去除类别型变量的空格...")
for col in df_with_desc.select_dtypes(include='object').columns:
    df_with_desc[col] = df_with_desc[col].astype(str).str.strip()
print(f"  ✓ 完成")

# 4. 重新分类（清理后）
numeric_features = df_with_desc.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df_with_desc.select_dtypes(include=['object']).columns.tolist()

print(f"\n最终分类:")
print(f"  - 数值型变量: {len(numeric_features)} 个")
print(f"  - 类别型变量: {len(categorical_features)} 个")
print(f"  - 总计: {len(numeric_features) + len(categorical_features)} 个")

In [ ]:
# Cell 5: 数值型变量质量分析
print("="*80)
print("步骤 3: 数值型变量质量分析")
print("="*80)

# 计算每个数值型变量的质量指标
quality_stats = []

for col in numeric_features:
    non_null = df_with_desc[col].notna().sum()
    null_count = df_with_desc[col].isna().sum()
    coverage = (non_null / len(df_with_desc)) * 100
    
    if non_null > 0:
        mean_val = df_with_desc[col].mean()
        std_val = df_with_desc[col].std()
        min_val = df_with_desc[col].min()
        max_val = df_with_desc[col].max()
    else:
        mean_val = std_val = min_val = max_val = np.nan
    
    quality_stats.append({
        '变量名': col,
        '非空数量': non_null,
        '缺失数量': null_count,
        '覆盖率%': coverage,
        '均值': mean_val,
        '标准差': std_val,
        '最小值': min_val,
        '最大值': max_val
    })

quality_df = pd.DataFrame(quality_stats)
quality_df = quality_df.sort_values('覆盖率%', ascending=False)

# 统计不同质量等级的变量
excellent = quality_df[quality_df['覆盖率%'] >= 95]
good = quality_df[(quality_df['覆盖率%'] >= 80) & (quality_df['覆盖率%'] < 95)]
fair = quality_df[(quality_df['覆盖率%'] >= 50) & (quality_df['覆盖率%'] < 80)]
poor = quality_df[quality_df['覆盖率%'] < 50]

print(f"\n数据质量分级:")
print(f"  ✓ 优秀 (覆盖率 >= 95%): {len(excellent)} 个变量")
print(f"  ○ 良好 (80% <= 覆盖率 < 95%): {len(good)} 个变量")
print(f"  △ 一般 (50% <= 覆盖率 < 80%): {len(fair)} 个变量")
print(f"  ✗ 较差 (覆盖率 < 50%): {len(poor)} 个变量")

print(f"\n前20个高质量数值型变量:")
print("="*80)
display(quality_df.head(20)[['变量名', '非空数量', '缺失数量', '覆盖率%', '均值', '标准差']])

In [ ]:
# Cell 6: 检查 XGBoost 关键特征
print("="*80)
print("步骤 4: XGBoost 模型关键特征检查")
print("="*80)

# 定义你们 XGBoost 模型中使用的关键特征
xgboost_key_features = [
    # 基础贷款信息
    "loan_amnt", "int_rate", "installment", "annual_inc", "dti",
    
    # 信用历史
    "delinq_2yrs", "mths_since_last_delinq", "num_accts_ever_120_pd",
    
    # FICO
    "fico_range_low", "fico_range_high",
    
    # 查询记录
    "inq_last_6mths", "mths_since_recent_inq",
    
    # 账户
    "open_acc", "total_acc", "mort_acc", "pub_rec",
    
    # 循环信贷
    "revol_bal", "revol_util", "total_rev_hi_lim",
    
    # 分期付款
    "total_bal_il", "il_util", "all_util",
    
    # 其他重要特征
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op",
    "num_rev_accts", "num_rev_tl_bal_gt_0", "num_actv_rev_tl",
    "pct_tl_nvr_dlq", "percent_bc_gt_75",
    "pub_rec_bankruptcies", "tax_liens"
]

# 检查这些特征在数据集中的存在性和质量
feature_check = []

for feature in xgboost_key_features:
    if feature in df_with_desc.columns:
        coverage = (df_with_desc[feature].notna().sum() / len(df_with_desc)) * 100
        status = "✓ 可用" if coverage >= 80 else "⚠ 质量较低"
        feature_check.append({
            '特征名': feature,
            '存在': '✓',
            '覆盖率%': f"{coverage:.2f}%",
            '状态': status
        })
    else:
        feature_check.append({
            '特征名': feature,
            '存在': '✗',
            '覆盖率%': 'N/A',
            '状态': '✗ 不存在'
        })

feature_check_df = pd.DataFrame(feature_check)

available = len(feature_check_df[feature_check_df['存在'] == '✓'])
missing = len(feature_check_df[feature_check_df['存在'] == '✗'])

print(f"\nXGBoost 关键特征检查结果:")
print(f"  - 可用特征: {available}/{len(xgboost_key_features)}")
print(f"  - 缺失特征: {missing}/{len(xgboost_key_features)}")

print(f"\n详细列表:")
display(feature_check_df)

In [ ]:
# Cell 7: 类别型变量分析
print("="*80)
print("步骤 5: 类别型变量分析")
print("="*80)

# 分析每个类别型变量
categorical_stats = []

for col in categorical_features:
    non_null = df_with_desc[col].notna().sum()
    unique_count = df_with_desc[col].nunique()
    coverage = (non_null / len(df_with_desc)) * 100
    
    # 获取最常见的值
    if non_null > 0:
        top_value = df_with_desc[col].value_counts().index[0] if unique_count > 0 else 'N/A'
        top_count = df_with_desc[col].value_counts().values[0] if unique_count > 0 else 0
    else:
        top_value = 'N/A'
        top_count = 0
    
    categorical_stats.append({
        '变量名': col,
        '非空数量': non_null,
        '覆盖率%': f"{coverage:.2f}%",
        '唯一值数量': unique_count,
        '最常见值': str(top_value)[:30],
        '最常见值数量': top_count
    })

categorical_df = pd.DataFrame(categorical_stats)

print(f"\n类别型变量总数: {len(categorical_features)}")
print(f"\n前20个类别型变量:")
display(categorical_df.head(20))

# 重点关注 XGBoost 模型中使用的类别型特征
xgb_categorical = ["term", "grade", "sub_grade", "emp_length", "home_ownership", 
                   "verification_status", "purpose", "application_type"]

print(f"\nXGBoost 类别型特征详情:")
print("="*80)
for col in xgb_categorical:
    if col in df_with_desc.columns:
        print(f"\n{col}:")
        print(f"  唯一值数量: {df_with_desc[col].nunique()}")
        print(f"  值分布:")
        value_counts = df_with_desc[col].value_counts().head(5)
        for val, count in value_counts.items():
            pct = (count / len(df_with_desc)) * 100
            print(f"    - {val}: {count:,} ({pct:.2f}%)")

In [ ]:
# Cell 8: 目标变量分析
print("="*80)
print("步骤 6: 目标变量 (loan_status) 分析")
print("="*80)

if 'loan_status' in df_with_desc.columns:
    print(f"\nloan_status 分布:")
    status_counts = df_with_desc['loan_status'].value_counts()
    
    for status, count in status_counts.items():
        pct = (count / len(df_with_desc)) * 100
        print(f"  - {status}: {count:,} ({pct:.2f}%)")
    
    # 检查是否有 Fully Paid 和 Charged Off
    if 'Fully Paid' in status_counts.index and 'Charged Off' in status_counts.index:
        fully_paid = status_counts['Fully Paid']
        charged_off = status_counts['Charged Off']
        total_relevant = fully_paid + charged_off
        
        print(f"\n用于建模的贷款状态:")
        print(f"  - Fully Paid: {fully_paid:,} ({fully_paid/total_relevant*100:.2f}%)")
        print(f"  - Charged Off: {charged_off:,} ({charged_off/total_relevant*100:.2f}%)")
        print(f"  - 总计: {total_relevant:,}")
        print(f"\n类别不平衡比例: {fully_paid/charged_off:.2f}:1 (Fully Paid:Charged Off)")
else:
    print("\n⚠ 警告: loan_status 列不存在")

In [ ]:
# Cell 9: 数据质量可视化
print("="*80)
print("步骤 7: 数据质量可视化")
print("="*80)

# 1. 数值型变量覆盖率柱状图（Top 30）
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 图1: 数值型变量覆盖率 (Top 30)
top_30_numeric = quality_df.head(30)
axes[0, 0].barh(range(len(top_30_numeric)), top_30_numeric['覆盖率%'], color='steelblue')
axes[0, 0].set_yticks(range(len(top_30_numeric)))
axes[0, 0].set_yticklabels(top_30_numeric['变量名'], fontsize=8)
axes[0, 0].set_xlabel('覆盖率 (%)', fontsize=10)
axes[0, 0].set_title('Top 30 数值型变量覆盖率', fontsize=12, fontweight='bold')
axes[0, 0].axvline(x=80, color='red', linestyle='--', alpha=0.5, label='80% 阈值')
axes[0, 0].legend()
axes[0, 0].invert_yaxis()

# 图2: 数据质量分级饼图
quality_counts = [len(excellent), len(good), len(fair), len(poor)]
quality_labels = [
    f'优秀 (≥95%)\n{len(excellent)}个',
    f'良好 (80-95%)\n{len(good)}个',
    f'一般 (50-80%)\n{len(fair)}个',
    f'较差 (<50%)\n{len(poor)}个'
]
colors = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c']
axes[0, 1].pie(quality_counts, labels=quality_labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('数值型变量质量分级', fontsize=12, fontweight='bold')

# 图3: loan_status 分布（如果存在）
if 'loan_status' in df_with_desc.columns:
    status_counts = df_with_desc['loan_status'].value_counts().head(10)
    axes[1, 0].bar(range(len(status_counts)), status_counts.values, color='coral')
    axes[1, 0].set_xticks(range(len(status_counts)))
    axes[1, 0].set_xticklabels(status_counts.index, rotation=45, ha='right', fontsize=8)
    axes[1, 0].set_ylabel('数量', fontsize=10)
    axes[1, 0].set_title('Loan Status 分布', fontsize=12, fontweight='bold')
    axes[1, 0].grid(axis='y', alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'loan_status 列不存在', 
                    ha='center', va='center', fontsize=12)
    axes[1, 0].set_title('Loan Status 分布', fontsize=12, fontweight='bold')

# 图4: desc 文本长度分布
desc_lengths = df_with_desc['desc'].astype(str).str.len()
axes[1, 1].hist(desc_lengths, bins=50, color='seagreen', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(desc_lengths.mean(), color='red', linestyle='--', 
                   linewidth=2, label=f'均值: {desc_lengths.mean():.0f}')
axes[1, 1].axvline(desc_lengths.median(), color='blue', linestyle='--', 
                   linewidth=2, label=f'中位数: {desc_lengths.median():.0f}')
axes[1, 1].set_xlabel('文本长度（字符数）', fontsize=10)
axes[1, 1].set_ylabel('频数', fontsize=10)
axes[1, 1].set_title('desc 字段文本长度分布', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../../data_quality_report.png', dpi=300, bbox_inches='tight')
print("\n✓ 图表已保存为 data_quality_report.png")
plt.show()

In [ ]:
# Cell 10: 保存清理后的数据
print("="*80)
print("步骤 8: 保存清理后的数据")
print("="*80)

output_file = '../../data/loan_with_desc.csv'

print(f"\n正在保存数据到 {output_file}...")
print(f"(这可能需要1-2分钟)\n")

df_with_desc.to_csv(output_file, index=False)

# 验证保存的文件
import os
file_size = os.path.getsize(output_file) / (1024 ** 2)  # MB

print(f"✓ 数据保存成功!")
print(f"\n文件信息:")
print(f"  - 文件名: {output_file}")
print(f"  - 文件大小: {file_size:.2f} MB")
print(f"  - 行数: {len(df_with_desc):,}")
print(f"  - 列数: {len(df_with_desc.columns)}")

# 保存数值型变量质量报告
quality_report_file = '../../numeric_features_quality_report.csv'
quality_df.to_csv(quality_report_file, index=False)
print(f"\n✓ 数值型变量质量报告已保存: {quality_report_file}")

# 保存类别型变量统计
categorical_report_file = '../../categorical_features_report.csv'
categorical_df.to_csv(categorical_report_file, index=False)
print(f"✓ 类别型变量报告已保存: {categorical_report_file}")

In [ ]:
# Cell 11: 生成最终数据质量摘要报告
print("="*80)
print("最终数据质量摘要报告")
print("="*80)

print(f"\n" + "="*80)
print("1. 数据集概览")
print("="*80)
print(f"原始数据集: {len(df):,} 行 × {len(df.columns)} 列")
print(f"清理后数据集: {len(df_with_desc):,} 行 × {len(df_with_desc.columns)} 列")
print(f"数据保留率: {len(df_with_desc)/len(df)*100:.2f}%")

print(f"\n" + "="*80)
print("2. 变量统计")
print("="*80)
print(f"数值型变量: {len(numeric_features)} 个")
print(f"  - 优秀质量 (覆盖率 ≥ 95%): {len(excellent)} 个")
print(f"  - 良好质量 (80% ≤ 覆盖率 < 95%): {len(good)} 个")
print(f"  - 一般质量 (50% ≤ 覆盖率 < 80%): {len(fair)} 个")
print(f"  - 较差质量 (覆盖率 < 50%): {len(poor)} 个")
print(f"\n类别型变量: {len(categorical_features)} 个")

print(f"\n" + "="*80)
print("3. XGBoost 模型就绪度")
print("="*80)
available_xgb_features = feature_check_df[feature_check_df['存在'] == '✓']
print(f"关键数值型特征可用: {len(available_xgb_features)}/{len(xgboost_key_features)}")
print(f"\n推荐用于 XGBoost 的高质量特征:")
high_quality_features = quality_df[quality_df['覆盖率%'] >= 90]['变量名'].tolist()
print(f"  - 总数: {len(high_quality_features)} 个")
print(f"  - 前20个: {', '.join(high_quality_features[:20])}")

print(f"\n" + "="*80)
print("4. OCEAN 特征提取就绪度")
print("="*80)
desc_lengths = df_with_desc['desc'].astype(str).str.len()
print(f"✓ desc 字段可用于 OCEAN 分析")
print(f"  - 样本数量: {len(df_with_desc):,}")
print(f"  - 平均文本长度: {desc_lengths.mean():.0f} 字符")
print(f"  - 中位数长度: {desc_lengths.median():.0f} 字符")
print(f"  - 适合语义分析: ✓")

print(f"\n" + "="*80)
print("5. 数据文件输出")
print("="*80)
print(f"✓ {output_file} ({file_size:.2f} MB)")
print(f"✓ {quality_report_file}")
print(f"✓ {categorical_report_file}")
print(f"✓ data_quality_report.png")

print(f"\n" + "="*80)
print("6. 下一步建议")
print("="*80)
print("✓ 1. 使用 loan_with_desc.csv 进行 OCEAN 特征提取")
print("✓ 2. 将 OCEAN 特征与现有数值型特征合并")
print("✓ 3. 使用合并后的特征训练 XGBoost 模型")
print("✓ 4. 对比有/无 OCEAN 特征的模型性能")

print(f"\n" + "="*80)
print("数据清理完成！")
print("="*80)